### **Requesitos para executar o script** 

Instalar as  bibliotecas:

* pandas 

* sqlite3

Documentações das bibliotecas:

* [pandas](https://pandas.pydata.org/)

* [sqlite3](https://docs.python.org/3/library/sqlite3.html)

In [ ]:
pip install pandas as pd

In [ ]:
pip install sqlite3

### **CASE 1** 

* Encontre a data da atividade e a pe_description das instalações com o nome STREET CHURROS  e com uma pontuação inferior a 95 pontos.

**Tabela: los_angeles_restaurant_health_inspections**

In [101]:
#Importação das bibliotecas

import pandas as pd
import sqlite3

In [ ]:
# Carregar o CSV para o  DataFrame do pandas
los_angeles_restaurant_health_inspections = pd.read_csv('./Dados/los_angeles_restaurant_health_inspections.csv', delimiter=';') 

# Conectar ao banco de dados SQLite em memória
conexao = sqlite3.connect(':memory:')

# Insere o DataFrame gerado em uma tabela no banco de dados
# A função 'to_sql' cria a tabela e insere os dados automaticamente
los_angeles_restaurant_health_inspections.to_sql('los_angeles_restaurant_health_inspections', conexao, if_exists='replace', index=False)

In [103]:
# Consulta SQL para selecionar os dados

consulta_sql = '''

    SELECT activity_date, pe_description
    FROM los_angeles_restaurant_health_inspections
    WHERE facility_name = 'STREET CHURROS'
      AND score < 95;
'''

# Executar a consulta e carregar o resultado em um DataFrame
resultado_query_case_1 = pd.read_sql_query(consulta_sql, conexao)

# Fecha a conexão com o banco de dados
conexao.close()

In [ ]:
resultado_query_case_1

### **CASE 2** 

* Escreva uma consulta que calcule a diferença entre os maiores salários encontrados
nos departamentos de marketing e engenharia. Exiba apenas a diferença absoluta
entre os salários.

**Tabelas: db_employee, db_dept**

In [ ]:
#Carregar o CSV para o  DataFrame do pandas
df_dept = pd.read_csv('./Dados/db_dept.csv', delimiter=';') 
df_employee = pd.read_csv('./Dados/db_employee.csv', delimiter=';')  

# Conectar ao banco de dados SQLite em memória
conexao = sqlite3.connect(':memory:')

# Insere o DataFrame gerado em uma tabela no banco de dados
# A função 'to_sql' cria a tabela e insere os dados automaticamente
df_dept.to_sql('db_dept', conexao, if_exists='replace', index=False)
df_employee.to_sql('db_employee', conexao, if_exists='replace', index=False)

In [106]:
# Consulta SQL para selecionar os dados

consulta_sql = '''
   SELECT 
    ABS(MAX(CASE WHEN d.department = 'marketing' THEN e.salary END) - 
    MAX(CASE WHEN d.department = 'engineering' THEN e.salary END)) AS salary_difference
FROM 
    db_employee e
JOIN 
    db_dept d ON e.department_id = d.id
WHERE 
    d.department IN ('marketing', 'engineering');
'''

# Executar a consulta e carregar o resultado em um DataFrame
resultado_query_case_2 = pd.read_sql_query(consulta_sql, conexao)

# Fecha a conexão com o banco de dados
conexao.close()

In [ ]:
resultado_query_case_2

### **CASE 3**

* Encontre os detalhes de cada cliente, independentemente de o cliente ter feito um pedido. Exiba o primeiro nome, o sobrenome e a cidade do cliente junto com os detalhes do pedido. Ordene os registros com base no primeiro nome do cliente e nos detalhes do pedido em ordem ascendente.

**Tabelas: customers, orders**

In [ ]:
# Carregar o CSV para o  DataFrame do pandas
df_customers = pd.read_csv('./Dados/customers.csv', delimiter=';')
df_orders = pd.read_csv('./Dados/orders.csv', delimiter=';') 

# Conectar ao banco de dados SQLite em memória
conexao = sqlite3.connect(':memory:')

# Insere o DataFrame gerado em uma tabela no banco de dados
# A função 'to_sql' cria a tabela e insere os dados automaticamente
df_customers.to_sql('customers', conexao, if_exists='replace', index=False)
df_orders.to_sql('orders', conexao, if_exists='replace', index=False)

In [109]:
# Consulta SQL para selecionar os dados

consulta_sql = '''
    SELECT 
    c.first_name,
    c.last_name,
    c.city,
    o.order_details,
    o.total_order_cost
FROM 
    customers c
LEFT JOIN 
    orders o ON c.id = o.cust_id
ORDER BY 
    c.first_name ASC, 
    o.order_details ASC;
'''

# Executar a consulta e carregar o resultado em um DataFrame
resultado_query_case_3 = pd.read_sql_query(consulta_sql, conexao)

# Fecha a conexão com o banco de dados
conexao.close()


In [ ]:
resultado_query_case_3

### **CASE 4**

* Você recebeu um conjunto de dados que fornece o número de usuários ativos por dia por conta premium. Uma conta premium terá uma entrada para cada dia em que for premium. No entanto, uma conta premium pode ser temporariamente descontada e considerada não paga, o que é
indicado por um valor de 0 na coluna final_price para um determinado dia. Descubra quantas contas premium pagas em um determinado dia ainda são premium e pagas 7 dias depois. Exiba a data, o número de contas premium e pagas naquele dia, e o número de quantas dessas contas ainda são premium e pagas 7 dias depois. Como você só tem dados para um período de 14 dias, inclua apenas as primeiras 7 datas disponíveis no resultado.

**Tabela: premium_accounts_by_day**


In [ ]:
# Carregar o CSV para o  DataFrame do pandas
premium_accounts_by_day = pd.read_csv('./Dados/premium_accounts_by_day.csv', delimiter=';')

# Conectar ao banco de dados SQLite em memória
conexao = sqlite3.connect(':memory:')

# Insere o DataFrame gerado em uma tabela no banco de dados
# A função 'to_sql' cria a tabela e insere os dados automaticamente
premium_accounts_by_day.to_sql('premium_accounts_by_day', conexao, if_exists='replace', index=False)

In [112]:
# Consulta SQL para selecionar os dados

consulta_sql = '''

WITH active_accounts AS (
    SELECT 
        entry_date,
        account_id,
        final_price
    FROM 
        premium_accounts_by_day
    WHERE 
        final_price > 0
),

-- Subconsulta para obter contas ativas 7 dias depois

accounts_seven_days_later AS (
    SELECT 
        a1.entry_date AS data_atual,
        COUNT(DISTINCT a1.account_id) AS contas_pagas_ativas,
        COUNT(DISTINCT a2.account_id) AS contas_pagas_ativas_sete_dias_depois
    FROM 
        active_accounts a1
    LEFT JOIN 
        active_accounts a2 
    ON 
        a1.account_id = a2.account_id 
        AND a2.entry_date = DATE(a1.entry_date, '+7 days')
    GROUP BY 
        a1.entry_date
)

-- Seleciona apenas as primeiras 7 datas disponíveis
SELECT 
    data_atual,
    contas_pagas_ativas,
    contas_pagas_ativas_sete_dias_depois
FROM 
    accounts_seven_days_later
ORDER BY 
    data_atual
LIMIT 7;

'''

# Executar a consulta e carregar o resultado em um DataFrame
resultado_query_case_4 = pd.read_sql_query(consulta_sql, conexao)

# Fecha a conexão com o banco de dados
conexao.close()

In [ ]:
resultado_query_case_4

### **CASE 5** 

* Encontre a taxa de retenção mensal de usuários para cada conta separadamente para dezembro de 2020 e janeiro de 2021. A taxa de retenção é a porcentagem de usuários ativos que uma conta retém durante um determinado período de tempo. Neste caso, assuma que o usuário é retido se ele/ela permanecer usando o aplicativo em qualquer mês futuro. Por exemplo, se um usuário esteve ativo em dezembro de 2020 e tem
atividade em qualquer mês futuro, considere-o retido para dezembro. Você pode assumir que todas as contas estão presentes em dezembro de 2020 e janeiro de 2021.
Sua saída deve conter o ID da conta e a taxa de retenção de janeiro de 2021 dividida pela taxa de retenção de dezembro de 2020.

**Tabela: sf_events**

In [ ]:
# Carregar o CSV para o  DataFrame do pandas
sf_events = pd.read_csv('Dados/sf_events.csv', delimiter=';')

# Converte a coluna 'date' para datetime
sf_events['date'] = pd.to_datetime(sf_events['date'], format='%d/%m/%Y')

# Conectar ao banco de dados SQLite em memória
conexao = sqlite3.connect(':memory:')

# Insere o DataFrame gerado em uma tabela no banco de dados
# A função 'to_sql' cria a tabela e insere os dados automaticamente
sf_events.to_sql('user_activity', conexao, index=False, if_exists='replace')

In [115]:
# Consulta SQL para selecionar os dados
consulta_sql = '''
WITH users_dec_2020 AS (
    SELECT DISTINCT user_id, account_id
    FROM user_activity
    WHERE date >= '2020-12-01' AND date < '2021-01-01'
),

users_jan_2021 AS (
    SELECT DISTINCT user_id, account_id
    FROM user_activity
    WHERE date >= '2021-01-01' AND date < '2021-02-01'
)

SELECT 
    a.account_id,
    COUNT(DISTINCT u_dec.user_id) AS total_users_dec_2020,
    COUNT(DISTINCT u_jan.user_id) AS retained_users_jan_2021,
    COUNT(DISTINCT u_jan.user_id) * 1.0 / NULLIF(COUNT(DISTINCT u_dec.user_id), 0) AS retention_rate_jan_2021,
    COUNT(DISTINCT u_jan.user_id) * 1.0 / NULLIF(COUNT(DISTINCT u_dec.user_id), 0) / NULLIF(COUNT(DISTINCT u_dec.user_id) * 1.0 / NULLIF(COUNT(DISTINCT u_dec.user_id), 0), 0) AS retention_ratio
FROM 
    users_dec_2020 u_dec
LEFT JOIN 
    users_jan_2021 u_jan ON u_dec.user_id = u_jan.user_id AND u_dec.account_id = u_jan.account_id
JOIN 
    (SELECT DISTINCT account_id FROM user_activity WHERE date >= '2020-12-01' AND date < '2021-02-01') a ON u_dec.account_id = a.account_id
GROUP BY 
    a.account_id;

'''

# Executar a consulta e carregar o resultado em um DataFrame
resultado_query_case_5 = pd.read_sql_query(consulta_sql, conexao)

# Fecha a conexão com o banco de dados
conexao.close()




In [ ]:
resultado_query_case_5

### **CASE 6** 

* Selecione o client_id mais popular com base na contagem do número de usuários que têm pelo menos 50% de seus eventos da seguinte lista: &#39;video call received&#39;, &#39;video call sent&#39;, &#39;voice call
received&#39;, &#39;voice call sent&#39;.

**Tabela: fact_events**

In [117]:
# Carregar o CSV para o  DataFrame do pandas
fact_events = pd.read_csv('./Dados/fact_events.csv', delimiter=';') 

# Conectar ao banco de dados SQLite em memória
conexao = sqlite3.connect(':memory:')

# Insere o DataFrame gerado em uma tabela no banco de dados
# A função 'to_sql' cria a tabela e insere os dados automaticamente
fact_events.to_sql('fact_events', conexao, if_exists='replace', index=False)

# Definir a lista de tipos de eventos
event_types = [
    'video call received',
    'video call sent',
    'voice call received',
    'voice call sent'
]

# Formatar a lista de tipos de eventos para SQL
event_types_sql = ', '.join(f"'{event}'" for event in event_types)


In [118]:
# Consulta SQL para selecionar os dados

consulta_sql = f"""
WITH UserEventCounts AS (
    SELECT user_id,
           COUNT(*) AS total_events,
           SUM(CASE WHEN event_type IN ({event_types_sql}) THEN 1 ELSE 0 END) AS filtered_events
    FROM fact_events
    GROUP BY user_id
),
FilteredUsers AS (
    SELECT user_id
    FROM UserEventCounts
    WHERE (filtered_events * 1.0 / total_events) >= 0.5
)
SELECT client_id, COUNT(DISTINCT user_id) AS user_count
FROM fact_events
WHERE user_id IN (SELECT user_id FROM FilteredUsers)
GROUP BY client_id
ORDER BY user_count DESC
LIMIT 1;
"""

# Executar a consulta e armazenar o resultado em um DataFrame
resultado_query_case_6 = pd.read_sql_query(consulta_sql, conexao)

# Fechar a conexão
conexao.close()

In [ ]:
resultado_query_case_6